# Doğrusal Regresyon Egzersizleri

50 adet Startup'ın araştırma ve geliştirmeye yönelik harcaması, yönetime yönelik harcaması, pazarlama harcaması, kazandıkları kar miktarı ve kuruldukları lokasyon bilgisi bulunmaktadır. Amaç kar miktarını tahmin etmektir. Bu bir sayısal tahmin problemidir ve bağımlı değişkenimiz "Profit".

Numpy, matplotlib.pyplot, pandas ve seaborn kütüphanelerini çekirdeğe dahil edelim.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Dizinde bulunan veri çerçevemizi startups değişkenine atayalım. startups değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
startups=pd.read_csv("../input/various-expenses-and-the-profits-of-50-startups/50_Startups.csv")
df=startups.copy()

İlk 5 gözlemini yazdıralım.

In [ ]:
df.head()

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()

Kaç gözlem ve öznitelikten oluştuğunu görüntüleyelim.

In [ ]:
df.shape

Veri setimiz 5 öznitelik ve 50 gözlemden oluşmaktadır.

In [ ]:
df.dtypes 

Eksik verileri kontrol edelim.

In [ ]:
df.isna().sum()

Korelasyon matrisi çizdirelim.

In [ ]:
df.corr()

R&D Spend ile Profit arasında güçlü ve pozitif yönde bir ilişki, Administration ile Profit arasında zayıf ve pozitif yönde bir ilişki, Marketing Spend ile Profit arasında güçlü
ve pozitif yönde bir ilişki olduğunu gözlemleyebiliriz. EN güçlü ilişki R&D Spend ile Profit arasındadır.

Seaborn ile korelasyon matrisinin ısı haritasını çizdirelim.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, linewidths=1, ax=ax, xticklabels=True, yticklabels=True);

R&D Spend ve Profit arasındaki korelasyonu daha iyi görebilmek için scatterplot çizdirelim.

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
sns.scatterplot(df["Profit"],df["R&D Spend"])

Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
plt.hist([df["Profit"], df["Administration"], df["Marketing Spend"], df["R&D Spend"]],
         color=['#0c457d','#d4aaff','#0ea7d8','#0ea7a3'],
         label=["Profit","Administration","Marketing Spend","R&D Spend"])

plt.legend() 
plt.show()

In [ ]:
sns.countplot(df["State"]);

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

State'a ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["State"].unique()

get_dummies yardımıyla State'a dair kategorik öznitelik çıkarımlarında bulunalım. Çünkü State'ların birbirine üstünlüğü yok, nominaller. Ordinal değil.

In [ ]:
df["State"]=pd.Categorical(df["State"])
dfDummies=pd.get_dummies(df["State"],  prefix="State")
dfDummies

In [ ]:
df=pd.concat([df, dfDummies], axis=1)
df.head()

State özniteliğini silip dummy olarak yaratılan State'lardan da birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df.drop(["State","State_New York"], axis=1, inplace=True)

In [ ]:
df.head()

Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
x=df.drop("Profit", axis=1)
y=df["Profit"]

Bağımlı ve bağımsız değişkenleri kontrol edelim.

In [ ]:
x

In [ ]:
y

Bu bağımlı ve bağımsız değişkenlerden train ve test olmak üzere 4 parça oluşturalım. Bunu yapmak için train_test_split kullanalım.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

4 parça değişkeni kontrol edelim.

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

LinearRegression'u çekirdeğe dahil edip modeli inşa edelim.

In [ ]:
lm=LinearRegression()

Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim.

In [ ]:
model=lm.fit(x_train,y_train)

Modele daha önce görmediği bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
y_pred=model.predict(x_test)
y_pred

Tahminleri ve gerçek değerleri bir veri çerçevesinde toplayıp üzerinde göz gezdirelim.

In [ ]:
test=pd.DataFrame(x_test).copy()
test["Profit"]=y_test
test["predicted_profit"]=y_pred
test.head(10)

sklearn bünyesinde barınan metrics'i çekirdeğe dahil edelim ve MAE, MSE, RMSE değerlerini görüntüleyelim.

In [ ]:
mean_absolute_error(test["Profit"], test["predicted_profit"])

In [ ]:
mean_squared_error(y_train, model.predict(x_train))

In [ ]:
np.sqrt(mean_squared_error(test["Profit"], test["predicted_profit"]))

Modelin R Squared değerini eğitim verileri üzerinden yazdıralım.

In [ ]:
r2_score(y_train,model.predict(x_train))

Dileyenler statsmodel kullanarak hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığına da bakabilir. Modelde bazı feature selection işlemleri yaparak tekrardan eğitip yeni sonuçlar mukayese edilebilir.

In [ ]:
import statsmodels.api as sm
x_train_lm = sm.add_constant(x_train)
lr_1 = sm.OLS(y_train, x_train_lm).fit()

In [ ]:
print(lr_1.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['Features'] = x_train.columns
vif['VIF'] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif
vif

Varyans Enflasyon Faktörü veya VIF , özellik değişkenlerinin birbirleriyle ne kadar ilişkili olduğunu belirten nicel bir değerdir. Doğrusal modelimizi test etmek son derece önemli bir parametredir. Formülü VIFşudur:

VIF = 1/(1-Ri²)

Yüksek bir VIF, ilişkili bağımsız değişkenin modeldeki diğer değişkenlerle yüksek düzeyde eş doğrusal olduğunu gösterir.

In [ ]:
X = x_train.drop('State_California', 1,)

x_train_lm = sm.add_constant(X)

lr_2 = sm.OLS(y_train, x_train_lm).fit()

print(lr_2.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X.drop('State_Florida', 1)
# Build a second fitted model
x_train_lm = sm.add_constant(X)
lr_3 = sm.OLS(y_train, x_train_lm).fit()

# Printing the summary of the model
print(lr_3.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X = X.drop('Administration', 1)
x_train_lm = sm.add_constant(X)

lr_4 = sm.OLS(y_train, x_train_lm).fit()
print(lr_4.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif